In [2]:
import torch
from PAINN_Calculator import MLCalculator
from mace.calculators import MACECalculator

In [51]:
calc=MLCalculator('Selftraining2.ckpt')
mace_calc=MACECalculator('mace-omat-0-medium.model',device='cuda',default_dtype='float32')

/fs-computility/ai4phys/wangzhongyao/Selftraining-MD/PAINN_Calculator.py:22: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  state_dict = torch.load(model_path)["state_dict"]
/fs-computility/ai4phys/wangzhongyao/miniconda3/envs/mace/lib/python3.11/site-packages/mace/calculators/mace.py:143: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  torch.load(f=model_path, map_location=device)


Using head default out of ['default']
Default dtype float32 does not match model dtype float64, converting models to float32.


In [53]:
atoms_list=torch.load('atoms_list.pt',weights_only=False)

In [54]:
idx_list,atoms_list = zip(*atoms_list)
idx_list=list(idx_list)
atoms_list=list(atoms_list)

In [55]:
import numpy as np
def get_rmse(target,predict):
    return np.sqrt(np.mean((target - predict) ** 2))

In [56]:
from tqdm import tqdm
forces_rmse_list=[]
mace_forces_rmse_list=[]
for atoms in tqdm(atoms_list):
    forces = calc.get_forces(atoms)
    forces_rmse = get_rmse(atoms.get_forces(), forces)
    forces_rmse_list.append(forces_rmse)
    mace_forces_rmse_list.append(get_rmse(atoms.get_forces(), mace_calc.get_forces(atoms)))

100%|██████████| 740/740 [00:34<00:00, 21.38it/s]


In [57]:
# 去掉forces_rmse_list中比mace_forces_rmse_list大的值
mask=np.where(np.array(forces_rmse_list) < np.array(mace_forces_rmse_list), True, False)
forces_rmse_list = np.array(forces_rmse_list)[mask]
mace_forces_rmse_list = np.array(mace_forces_rmse_list)[mask]
idx_list = np.array(idx_list)[mask]


In [70]:
import matplotlib.pyplot as plt
import scienceplots

# 1. 首先应用你的样式表
# 'no-latex' 很重要，它告诉 scienceplots 使用 Matplotlib 的原生文本渲染引擎，
# 这样我们才能通过 rcParams 控制字体。
plt.style.use(['science', 'nature', 'no-latex'])

# 2. 在应用样式后，立刻用你自己的设置来覆盖字体相关的参数
# 这是最关键的一步
plt.rcParams.update({
    "font.family": "sans-serif",  # 声明使用 sans-serif 字体族
    "font.sans-serif": ['Tahoma', 'DejaVu Sans', 'Lucida Grande', 'Verdana']  # 提供一个 sans-serif 字体列表
})

# 3. 现在开始绘图，后续的绘图指令都会遵循上面更新后的 rcParams
plt.figure(figsize=(10, 6))
plt.plot(idx_list, forces_rmse_list, label='Student model RMSE', marker='o', markersize=3)
plt.plot(idx_list, mace_forces_rmse_list, label='Teacher model RMSE', marker='x', markersize=3)

plt.xlabel('MD Step / ps') # 注意单位里的斜杠，建议用 ps
plt.ylabel(r'RMSE / eV /$\AA$') # 使用LaTeX格式化单位，效果更好
plt.title('RMSE of Forces from Student and Teacher Models')
plt.legend()
plt.tight_layout() # 建议添加，可以自动调整布局，防止标签重叠

# 4. 保存图像
plt.savefig('rmse_sans_serif.png', dpi=600)

# 建议在脚本末尾关闭图形，释放内存
plt.close()

print("图像已保存为 rmse_sans_serif.png")


图像已保存为 rmse_sans_serif.png
